<a href="https://colab.research.google.com/github/Ranjankraj/Shala/blob/main/multiclass_20_ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip  "/content/20_newsgroups.zip"

In [ ]:
!unzip -u "/content/20_newsgroups.zip" -d "/content/Internship/"

In [57]:
!ls

20_newsgroups.zip  drive  Internship  sample_data


In [58]:
from os import listdir
from os.path import isfile, join
import string

In [64]:
my_path = "/content/Internship/20_newsgroups"

#creating a list of folder names to make valid pathnames later
folders = [f for f in listdir(my_path)]

In [65]:
folders

['rec.sport.hockey',
 'comp.os.ms-windows.misc',
 'alt.atheism',
 'sci.crypt',
 'comp.sys.ibm.pc.hardware',
 'sci.med',
 'sci.electronics',
 'comp.sys.mac.hardware',
 'misc.forsale',
 'comp.windows.x',
 'talk.religion.misc',
 'rec.autos',
 'talk.politics.misc',
 'rec.sport.baseball',
 'soc.religion.christian',
 'talk.politics.mideast',
 'rec.motorcycles',
 'comp.graphics',
 'sci.space',
 'talk.politics.guns']

In [66]:
print(len(folders))

20


In [67]:
#function to remove incompatible files
import os
import sklearn.datasets
def find_incompatible_files(path):
    """
    Finds the filenames that are incompatible with `CountVectorizer`. These files are usually not compatible with UTF8!
    parameter `path` is the absolute or relative path of the training data's root directory.
    returns a list of strings.
    """

    count_vector = sklearn.feature_extraction.text.CountVectorizer()
    files = sklearn.datasets.load_files(path)
    num = []
    for i in range(len(files.filenames)):
        try:
            count_vector.fit_transform(files.data[i:i + 1])
        except UnicodeDecodeError:
            num.append(files.filenames[i])
            os.remove(files.filenames[i])
        except ValueError:
            pass

    return num


In [63]:
#Removing incompatible files
val = find_incompatible_files(my_path)

In [68]:
#printing no. of incompatible files removed

print(len(val))

73


In [69]:
#creating a 2D list to store list of all files in different folders

files = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])

In [70]:
sum(len(files[i]) for i in range(20))

19924

In [71]:
pathname_list = []
for fo in range(len(folders)):
    for fi in files[fo]:
        pathname_list.append(join(my_path, join(folders[fo], fi)))

In [72]:
len(pathname_list)

19924

In [73]:
#making an array containing the classes each of the documents belong to

Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)

In [74]:
len(Y)

19924

In [101]:
#function for cleaning and reading data

import collections
import os.path
import glob
import nltk
from nltk.corpus import stopwords

def cleanDoc(doc):
    tokens = nltk.WordPunctTokenizer().tokenize(doc)
    # stop_words = set(stopwords.words('english'))
    # tokens = remove_stopwords(tokens)
    clean = [token.lower() for token in tokens if len(token) > 3 and token.isalpha()]
    # clean = [word for word in clean if not word in stopwords]
    return clean


def ps(doc):
      f = open(doc)
      data= f.read()
      words = cleanDoc(data)
      return words


#Storing list of all the words

list_of_all_words = []

for i in range(len(pathname_list)):
  for file in glob.glob(pathname_list[i]):
      
      words = ps(file)
      for word in words:
        list_of_all_words.append(word)



In [102]:
import numpy as np

np_list_of_all_words = np.asarray(list_of_all_words)

words, counts = np.unique(np_list_of_all_words, return_counts=True)
len(words)

95960

In [103]:
def count(word):
  d_fq = []
  for i in range(len(pathname_list)):
    for file in glob.glob(pathname_list[i]):

        f = open(file)
        data= f.read()
        words = cleanDoc(data)
        if word in words:
          d_fq.append(pathname_list[i])
  return len(d_fq)

In [ ]:
#dictionary will contain unique words mapped with their document frequency
d_words = {}

for word in words: d_words[word] = count(word)

In [ ]:
print(d_words)

In [ ]:
#Sorting dictionary
import operator

sorted_d = dict( sorted(d_words.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
#choosing 500 to 1000 words by document frequency
dict_items = sorted_d.keys()

f_t = list(dict_items)[500:1001]

print(f_t)

In [ ]:
#We can loop over the below function to find Probability(word|class)

def prob(word,class):
  d_fq = []
  
  for folder_name in folders:
    if class == folder_name:
      files = []
      folder_path = join(my_path, folder_name)
      files.append([f for f in listdir(folder_path)])
      pathname_list = []
      for fi in files[fo]:
          pathname_list.append(join(my_path, join(folders[fo], fi)))
      for i in range(len(pathname_list)):
        for file in glob.glob(pathname_list[i]):

          f = open(file)
          data= f.read()
          words = cleanDoc(data)
          if word in words:
            d_fq.append(pathname_list[i])

          return ((len(d_fq))/(len(pathname_list)))


w = 'article'
c = 'alt.atheism'

prob(w,c)

In [104]:
from sklearn.model_selection import train_test_split

In [105]:
do_train, do_test, Y_train, Y_test = train_test_split(pathname_list, Y, random_state=0, test_size=0.25)

In [106]:
def flatten(list):
    new_list = []
    for i in list:
        for j in i:
            new_list.append(j)
    return new_list

In [107]:
list_of_words = []

for document in do_train:
        list_of_words.append((ps(document)))

In [108]:
len(list_of_words)

14943

In [110]:
import numpy as np
np_list_of_words = np.asarray(flatten(list_of_words))


In [111]:
#finding the number of unique words that we have extracted from the documents

words, counts = np.unique(np_list_of_words, return_counts=True)
len(words)

84071

In [112]:
#sorting the unique words according to their frequency in all the documents

freq, wrds = (list(i) for i in zip(*(sorted(zip(counts, words), reverse=True))))

In [115]:
#deciding the no. of words to use as feature. took n = 1000 to 5000

n = 5001
features = wrds[1000:n]

In [116]:
#creating a dictionary for train that contains each document's vocabulary and ocurence of each word of the vocabulary 

dictionary = {}
doc_num = 1
for doc_words in list_of_words:
    #print(doc_words)
    np_doc_words = np.asarray(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary[doc_num] = {}
    for i in range(len(w)):
        dictionary[doc_num][w[i]] = c[i]
    doc_num = doc_num + 1

In [117]:
#now we make a 2D array having the frequency of each word of our feature set in each individual documents

X_train = []
for k in dictionary.keys():
    row = []
    for f in features:
        if(f in dictionary[k].keys()):
            #if word f is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(dictionary[k][f]) 
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    X_train.append(row)

In [118]:
#we convert the X and Y into np array for concatenation and conversion into dataframe

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

In [119]:
len(X_train)

14943

In [120]:
len(Y_train)

14943

In [122]:
list_of_words_test = []

for document in do_test:
        list_of_words_test.append(ps(document))

In [123]:
#creating a dictionary for test that contains each document's vocabulary and ocurence of each word of the vocabulary 

dictionary_test = {}
doc_num = 1
for doc_words in list_of_words_test:
    #print(doc_words)
    np_doc_words = np.asarray(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary_test[doc_num] = {}
    for i in range(len(w)):
        dictionary_test[doc_num][w[i]] = c[i]
    doc_num = doc_num + 1

In [124]:
#now we make a 2D array having the frequency of each word of our feature set in each individual documents

X_test = []
for k in dictionary_test.keys():
    row = []
    for f in features:
        if(f in dictionary_test[k].keys()):
            #if word f is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(dictionary_test[k][f]) 
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    X_test.append(row)

In [125]:
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

In [126]:
len(X_test)

4981

In [127]:
len(Y_test)

4981

In [128]:
#fitting the model 

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [129]:
Y_predict = clf.predict(X_test)

In [130]:
#accuracy for test data
clf.score(X_test, Y_test)

0.7751455531017868

In [135]:
#printing report on test
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(Y_test, Y_predict))

                          precision    recall  f1-score   support

             alt.atheism       0.74      0.76      0.75       251
           comp.graphics       0.66      0.66      0.66       267
 comp.os.ms-windows.misc       0.65      0.64      0.65       256
comp.sys.ibm.pc.hardware       0.60      0.65      0.63       233
   comp.sys.mac.hardware       0.71      0.71      0.71       267
          comp.windows.x       0.79      0.70      0.74       245
            misc.forsale       0.71      0.68      0.69       249
               rec.autos       0.73      0.82      0.77       236
         rec.motorcycles       0.81      0.88      0.84       234
      rec.sport.baseball       0.87      0.85      0.86       250
        rec.sport.hockey       0.84      0.96      0.90       228
               sci.crypt       0.88      0.92      0.90       239
         sci.electronics       0.73      0.69      0.71       272
                 sci.med       0.94      0.85      0.89       246
         

In [132]:
Y_predict_tr = clf.predict(X_train)

In [133]:
#accuracy for training data

clf.score(X_train, Y_train)

0.8584621561935354

In [137]:
#printing report on training set
print(classification_report(Y_train, Y_predict_tr))

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.87      0.83       742
           comp.graphics       0.80      0.78      0.79       728
 comp.os.ms-windows.misc       0.80      0.83      0.81       744
comp.sys.ibm.pc.hardware       0.78      0.78      0.78       762
   comp.sys.mac.hardware       0.79      0.83      0.81       718
          comp.windows.x       0.89      0.83      0.86       751
            misc.forsale       0.82      0.82      0.82       747
               rec.autos       0.86      0.89      0.87       760
         rec.motorcycles       0.88      0.93      0.91       766
      rec.sport.baseball       0.91      0.93      0.92       743
        rec.sport.hockey       0.91      0.96      0.94       766
               sci.crypt       0.92      0.92      0.92       760
         sci.electronics       0.85      0.84      0.85       722
                 sci.med       0.95      0.87      0.90       751
         